In [4]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [5]:
years = range(2024, 2029)
results = []

for year in years:
    baseline = Microsimulation()
    baseline_person = baseline.calculate("household_net_income", period=year, map_to="household")

    reform = Reform.from_dict({
      "gov.contrib.states.ny.wftc.amount.max": {
        "2024-01-01.2024-12-31": 550,
        "2025-01-01.2025-12-31": 800,
        "2026-01-01.2026-12-31": 1000,
        "2027-01-01.2027-12-31": 1200,
        "2028-01-01.2035-12-31": 1600
      },
      "gov.contrib.states.ny.wftc.amount.min": {
        "2024-01-01.2024-12-31": 0,
        "2025-01-01.2100-12-31": 100
      },
      "gov.contrib.states.ny.wftc.child_age_threshold": {
        "2024-01-01.2026-12-31": 17,
        "2027-01-01.2028-12-31": 18
      },
      "gov.contrib.states.ny.wftc.eitc.match": {
        "2024-01-01.2024-12-31": 0.25,
        "2025-01-01.2025-12-31": 0.2,
        "2026-01-01.2026-12-31": 0.15,
        "2027-01-01.2027-12-31": 0.1,
        "2028-01-01.2028-12-31": 0
      },
      "gov.contrib.states.ny.wftc.in_effect": {
        "2024-01-01.2100-12-31": True
      },
      "gov.contrib.states.ny.wftc.reduction.married[1].threshold": {
        "2024-01-01.2026-12-31": 110000,
        "2027-01-01.2027-12-31": 90000,
        "2028-01-01.2028-12-31": 50000,
      },
      "gov.contrib.states.ny.wftc.reduction.single[1].threshold": {
        "2024-01-01.2024-12-31": 75000,
        "2025-01-01.2025-12-31": 65000,
        "2026-01-01.2026-12-31": 55000,
        "2027-01-01.2027-12-31": 45000,
        "2028-01-01.2028-12-31": 25000,
      },
       "gov.contrib.states.ny.wftc.exemptions.child_age_threshold":{
        "2024-01-01.2024-12-31": 0,
        "2025-01-01.2026-12-31": 17,
           
       }
          
    }, country_id="us")
    reformed = Microsimulation(reform=reform)
    reformed_person = reformed.calculate("household_net_income", period=year, map_to="household")

    difference_person = (reformed_person - baseline_person).sum()
    results.append({"year": year, "difference": difference_person})

df = pd.DataFrame(results)

In [ ]:
import plotly.express as px
from policyengine_core.charts import *

# Create a new column in the DataFrame for the rounded values in billions
df['difference_billion'] = (df['difference'] / 1e9)

fig = px.bar(df, x='year', y='difference', title='New York Working Families Tax Credit costs over time', labels={'difference': 'Cost ($bn)', 'year': 'Year'}, text='difference_billion')
fig.update_xaxes(tickmode='linear', tick0=2024, dtick=1)
fig.update_traces(texttemplate='$%{text:.1f}B', textposition='outside')
fig = format_fig(fig)
fig.show()